In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys # button이 아닌 element도 클릭이 가능하게 만들어주는 모듈
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import urllib.request

In [3]:
product_df = pd.read_csv('./product_table.csv', encoding = 'latin-1')
product_df.head(5)

,product_id,product_name,price,detail,image
0,1,Chocolate Sandwich Cookies,NaN,NaN,NaN
1,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,NaN,NaN,NaN
2,8,Cut Russet Potatoes Steam N' Mash,NaN,NaN,NaN
3,10,Sparkling Orange Juice & Prickly Pear Beverage,NaN,NaN,NaN
4,12,Chocolate Fudge Layer Cake,NaN,NaN,NaN


In [17]:
browser = webdriver.Chrome()
url = 'https://www.amazon.com/'
browser.get(url)
#창 최대화
browser.maximize_window()


C:\Users\psm69\AppData\Local\Temp\ipykernel_8088\1954891913.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("C:/Users/psm69/Desktop/프로그래밍/Python/Project/브로컬리&스누피/크롤링/chromedriver.exe")


In [5]:
import os
#폴더 확인 후 없으면 폴더 생성
img_folder = './product_img'

if not os.path.isdir(img_folder):
    os.mkdir(img_folder)

In [61]:
p_price = []
img_name = []

wait = WebDriverWait(browser,4)

for p_name in product_df['product_name']:
    name = ''
    for s in p_name:
        #폴더에 들어갈 수 없는 문자들 제외
        if s not in ['?', '/', '~', '!', '@', '#', '$', '%', '^', '*', '\\', "'"]:
            name += s

    p_name = name

    browser.implicitly_wait(10)
    search_window = browser.find_element(By.ID,'twotabsearchtextbox')
    search_window.clear()

    time.sleep(0.5)
    
    #검색시 오류뜨는 경우를 대비하기 위해
    try:
        search_window.send_keys(p_name + '\n')

        #가격 정보
        try:
            price = wait.until(browser.find_elements(By.CLASS_NAME, 'a-price-whole'))
            fraction = browser.find_elements(By.CLASS_NAME, 'a-price-fraction')
            p_price.append(price[0].text+'.'+fraction[0].text)           
        except:
            #검색 정보 없으면 0 달러
            p_price.append('0')

        #이미지 정보
        try:
            img = browser.find_elements(By.CLASS_NAME, 's-image')
            link = img[0].get_attribute('src')
            urllib.request.urlretrieve(link, "./product_img/"+p_name+".jpg")
            img_name.append(p_name+".jpg")
        except:
            img_name.append('NaN')
    except:
        p_price.append('0')
        img_name.append('NaN')

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00346463+2188387]
	Ordinal0 [0x002DE461+1762401]
	Ordinal0 [0x001F3D78+802168]
	Ordinal0 [0x001E7210+750096]
	Ordinal0 [0x001E675A+747354]
	Ordinal0 [0x001E5D3F+744767]
	Ordinal0 [0x001E4C28+740392]
	Ordinal0 [0x001E5228+741928]
	Ordinal0 [0x001EF153+782675]
	Ordinal0 [0x001F9FBB+827323]
	Ordinal0 [0x001FD310+840464]
	Ordinal0 [0x001E54F6+742646]
	Ordinal0 [0x001F9BF3+826355]
	Ordinal0 [0x0024CF6D+1167213]
	Ordinal0 [0x0023C5F6+1099254]
	Ordinal0 [0x00216BE0+945120]
	Ordinal0 [0x00217AD6+948950]
	GetHandleVerifier [0x005E71F2+2712546]
	GetHandleVerifier [0x005D886D+2652765]
	GetHandleVerifier [0x003D002A+520730]
	GetHandleVerifier [0x003CEE06+516086]
	Ordinal0 [0x002E468B+1787531]
	Ordinal0 [0x002E8E88+1805960]
	Ordinal0 [0x002E8F75+1806197]
	Ordinal0 [0x002F1DF1+1842673]
	BaseThreadInitTh

In [ ]:
product_df['price'] = p_price
product_df['image'] = img_name

product_df.to_csv('./product_table.csv', index = False)